# Laboratorio de regresión - 4

|                |   |
:----------------|---|
| **Nombre**     Dayanni Godoy Rosales|   |
| **Fecha**     09/09/2024 |   |
| **Expediente*749206* |   |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente. 

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

Compara los resultados de los parámetros y sus *p-values*, y los $R^2$ resultantes.

In [111]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import statsmodels.api as sm
from scipy import stats 

In [30]:
data = pd.read_csv('Advertising.csv')
data

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9
...,...,...,...,...,...
195,196,38.2,3.7,13.8,7.6
196,197,94.2,4.9,8.1,9.7
197,198,177.0,9.3,6.4,12.8
198,199,283.6,42.0,66.2,25.5


In [56]:
x1 = data["TV"].values.reshape(-1,1)
x2 = data["radio"].values.reshape(-1,1)
x3 = data["newspaper"].values.reshape(-1,1)
y = data["sales"]

In [58]:
n = len(y)
ones = np.ones([n,1])
X = np.hstack([ones,x1,x2,x3])

## Sin penalización

In [61]:
ols = sm.OLS(y, X)
results = ols.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sales   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     570.3
Date:                Thu, 11 Sep 2025   Prob (F-statistic):           1.58e-96
Time:                        10:41:22   Log-Likelihood:                -386.18
No. Observations:                 200   AIC:                             780.4
Df Residuals:                     196   BIC:                             793.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.9389      0.312      9.422      0.000       2.324       3.554
x1             0.0458      0.001     32.809      0.000       0.043       0.049
x2             0.1885      0.009     21.893      0.000       0.172       0.206
x3            -0.0010      0.006     -0.177      0.860      -0.013       0.011
==============================================================================
Omnibus:                       60.414   Durbin-Watson:                   2.084
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              151.241
Skew:                          -1.327   Prob(JB):                     1.44e-33
Kurtosis:                       6.332   Cond. No.                         454.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [63]:
p_values = results.pvalues
print(p_values)

const    1.267295e-17
x1       1.509960e-81
x2       1.505339e-54
x3       8.599151e-01
dtype: float64


## Con penalización L2

In [143]:
from sklearn.linear_model import Ridge

In [145]:
ridge = Ridge(alpha=0.5)
ridge.fit(X, y)
r2_ridge= r2_score(y, ridge.predict(X))
print("Ridge Intercept:" + str(ridge.intercept_))
print("Ridge Coeficientes:" + str(ridge.coef_))
print("Ridge R2:" + str(r2_ridge))

Ridge Intercept:2.93892841431542
Ridge Coeficientes:[ 0.          0.04576464  0.18852756 -0.00103689]
Ridge R2:0.8972106381360829


In [147]:
n = len(y)
p = 4
y_pred = ridge.predict(X)
residuals = y - y_pred
# Suma de cuadrados residual
RSS = np.sum(residuals**2)  
RSS 

556.8252631344158

In [169]:
RSE = np.sqrt(RSS / (n - p))
RSE

1.685510373766222

In [175]:
sigma2 = RSS / (n - p)
var_beta = np.linalg.inv(X.T @ X) * sigma2
std_beta = np.sqrt(np.diag(var_beta))
std_beta

array([0.31190824, 0.0013949 , 0.00861123, 0.00587101])

In [177]:
# Estadísticos t
t_stats = ridge.coef_ / std_beta
t_stats

array([ 0.        , 32.80862378, 21.89321037, -0.17661247])

In [179]:
# Grados de libertad
df = n - p

# p-values bilaterales
p_values = [2 * (1 - stats.t.cdf(np.abs(t), df)) for t in t_stats]
p_values

[1.0, 0.0, 0.0, 0.8599951606746024]

## Con penalización L1

In [50]:
from sklearn.linear_model import Lasso

In [190]:
lasso = Lasso(alpha=0.5)
lasso.fit(X, y)
r2_lasso= r2_score(y, lasso.predict(X))
print("Lasso Intercept:", lasso.intercept_)
print("Lasso Coeficientes:", lasso.coef_)
print("Lasso R2:", r2_lasso)

Lasso Intercept: 2.980656625064583
Lasso Coeficientes: [ 0.          0.04570812  0.18572931 -0.        ]
Lasso R2: 0.8971515892246072


In [192]:
n = len(y)
p = 4
y_pred = lasso.predict(X)
residuals = y - y_pred
# Suma de cuadrados residual
RSS = np.sum(residuals**2)  
RSS 

557.1451398714054

In [194]:
RSE = np.sqrt(RSS / (n - p))
RSE

1.685994437784972

In [196]:
sigma2 = RSS / (n - p)
var_beta = np.linalg.inv(X.T @ X) * sigma2
std_beta = np.sqrt(np.diag(var_beta))
std_beta

array([0.31199781, 0.0013953 , 0.00861371, 0.0058727 ])

In [200]:
# Estadísticos t
t_stats = lasso.coef_ / std_beta
t_stats

array([ 0.        , 32.75869572, 21.56206482, -0.        ])

In [202]:
# Grados de libertad
df = n - p

# p-values bilaterales
p_values = [2 * (1 - stats.t.cdf(np.abs(t), df)) for t in t_stats]
p_values

[1.0, 0.0, 0.0, 1.0]

## Comparación de parámetros, p-values y R2

In [213]:
import pandas as pd

# Resultados OLS
ols_params = results.params.values        
ols_pvalues = results.pvalues.values
ols_r2 = results.rsquared

# Resultados Ridge
ridge_params = [ridge.intercept_] + list(ridge.coef_[1:])  # quitamos el coef de la constante extra
ridge_pvalues = p_values[:4] 
ridge_r2 = r2_ridge

# Resultados Lasso
lasso_params = [lasso.intercept_] + list(lasso.coef_[1:])
lasso_pvalues = p_values[:4]
lasso_r2 = r2_lasso

# Tabla comparativa
df_comp = pd.DataFrame({
    "OLS_coef": ols_params,
    "OLS_pval": ols_pvalues,
    "Ridge_coef": ridge_params,
    "Ridge_pval": ridge_pvalues,
    "Lasso_coef": lasso_params,
    "Lasso_pval": lasso_pvalues
}, index=["Intercepto", "TV", "Radio", "Newspaper"])

print(df_comp)

print("R2 sin penalización:", round(ols_r2,3),
      "R2 Ridge:", round(ridge_r2,3),
      "R2 Lasso:", round(r2_lasso,3))

            OLS_coef      OLS_pval  Ridge_coef  Ridge_pval  Lasso_coef  \
Intercepto  2.938889  1.267295e-17    2.938928         1.0    2.980657   
TV          0.045765  1.509960e-81    0.045765         0.0    0.045708   
Radio       0.188530  1.505339e-54    0.188528         0.0    0.185729   
Newspaper  -0.001037  8.599151e-01   -0.001037         1.0   -0.000000   

            Lasso_pval  
Intercepto         1.0  
TV                 0.0  
Radio              0.0  
Newspaper          1.0  
R2 sin penalización: 0.897 R2 Ridge: 0.897 R2 Lasso: 0.897


- Parámetros: Ridge tiende a encoger coeficientes pero no los elimina, por eso Newspaper sigue con valor casi igual a sin penalización. Newspaper se volvió exactamente 0 porque Lasso "seleccionó variables" y eliminó Newspaper del modelo.
- P-values: Debo tener un error porque si en Lasso ya no está Newspaper, por qué su p-value sería 1?
-  R2: Igual en los 3 casos.